## Week 2 Practical Assignment: Exploring Real-World Networks
# Analysis of the SNAP YouTube Social Network
## Course: Model Based Decisions (2025)

In [ ]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter
import seaborn as sns
import numpy as np
import gzip
import os
from collections import defaultdict
import seaborn as sns
from matplotlib.patches import Patch
import warnings
import math
import collections

warnings.filterwarnings('ignore')

# Set matplotlib to display plots inline
%matplotlib inline

# Set style for better looking plots
plt.style.use('ggplot')
sns.set_palette("husl")

seed = 2025 # Random seed for reproducible model generation

In [12]:

# #################################################################################################### #
# Network Visualization Suite Class (Adapted for YouTube Analysis from network_visualisation_suite.py) #
# #################################################################################################### #
class NetworkVisualizationSuite:
    """
    Comprehensive network analysis suite, adapted for N > 1M nodes.
    Uses sampling for path-dependent metrics (Betweenness, Closeness, Avg Path Length) 
    to ensure execution.
    """
    
    def __init__(self):
        """Initialize the suite."""
        self.networks = {} # Dictionary to store the networks
        self.network_stats = {} # Dictionary to store the network statistics
        self.youtube_url = "https://snap.stanford.edu/data/com-youtube.ungraph.txt.gz"
        self.real_network_key = 'YouTube'
        # Sampling size for all path-dependent metrics (L, Betweenness, Closeness)
        self.sampling_k = 1000 
        
    def load_youtube_network(self):
        """Load YouTube social network from compressed SNAP dataset URL and return LCC."""
        print(f"--- 1. Data Loading ---")
        # Load the dataset using pandas
        youtube_df = pd.read_csv(
            self.youtube_url,
            compression="gzip",
            sep="\t",
            comment="#",
            names=["start_node", "end_node"],
        )
        
        # Create the graph object G from the pandas edgelist 
        G = nx.from_pandas_edgelist(youtube_df, "start_node", "end_node")
        
        # Check connectivity and get LCC (which for YouTube is the entire graph)
        if nx.is_connected(G):
            G_lcc = G.copy()
            print("Graph is fully connected. Proceeding with the full graph.")
        else:
            # Fallback for LCC calculation
            largest_cc_nodes = max(nx.connected_components(G), key=len)
            G_lcc = G.subgraph(largest_cc_nodes).copy()

        print(f"Loaded YouTube network: {G_lcc.number_of_nodes():,} nodes, {G_lcc.number_of_edges():,} edges\n")
        self.networks[self.real_network_key] = G_lcc 
        return G_lcc
    
    def generate_theoretical_networks(self, G_lcc):
        """Generate theoretical network models (ER, WS, BA) for comparison."""
        
        n = G_lcc.number_of_nodes()
        e = G_lcc.number_of_edges()
        avg_deg = (2 * e) / n
        
        print(f"--- 2. Model Generation ---")
        
        # 1. Erdős-Rényi (ER) Model 
        p_er = avg_deg / (n - 1)
        er_graph = nx.erdos_renyi_graph(n, p_er, seed=seed) 
        self.networks = er_graph # Corrected dictionary assignment
        
        # 2. Watts-Strogatz (small-world) 
        k = max(4, int(round(avg_deg))) 
        k = k if k % 2 == 0 else k + 1 # Ensure k is even
        k_ws = k 
        p_ws = 0.1 
        ws_graph = nx.watts_strogatz_graph(n, k_ws, p_ws, seed=seed)
        self.networks = ws_graph # Corrected dictionary assignment
        
        # 3. Barabási-Albert (scale-free) 
        m = max(2, int(round(avg_deg / 2))) # m ≈ <k> / 2
        m_ba = m
        ba_graph = nx.barabasi_albert_graph(n, m_ba, seed=seed)
        self.networks = ba_graph # Corrected dictionary assignment
        
        print(f"Theoretical networks generated successfully (ER p={p_er:.8f}, WS k={k_ws}, BA m={m_ba})!\n")

    def _approximate_avg_path_length(self, G, k):
        """Approximates Average Shortest Path Length by sampling k source nodes."""
        nodes = list(G.nodes())
        sampled_nodes = np.random.choice(nodes, min(len(nodes), k), replace=False)
        total_path_length = 0
        num_paths = 0
        
        for source in sampled_nodes:
            # Calculate shortest paths from one source to all others
            try:
                length = nx.shortest_path_length(G, source=source)
            except nx.NetworkXNoPath:
                # Skip if the sampled component is somehow disconnected
                continue 
            
            # Sum path lengths and count valid paths
            for target, path_len in length.items():
                if source!= target:
                    total_path_length += path_len
                    num_paths += 1
                    
        if num_paths > 0:
            return total_path_length / num_paths
        return 0.0 # Return 0.0 or inf if path computation fails

    def analyze_network_properties(self):
        """
        Computes network metrics. Uses sampling for Avg. Path Length due to N > 1M.
        """
        print("--- 3. Analyzing Network Properties ---")
        
        for name, G in self.networks.items():
            print(f"Analyzing {name} network...")
            
            stats = {}
            stats['nodes'] = G.number_of_nodes()
            stats['edges'] = G.number_of_edges()
            stats['avg_degree'] = np.mean([d for n, d in G.degree()])
            
            # Clustering, Assortativity (All feasible exactly)
            stats['avg_clustering'] = nx.average_clustering(G)
            stats['assortativity'] = nx.degree_assortativity_coefficient(G) 
            
            # Path lengths (Approximated via Sampling)
            # Use a slightly smaller k for the models to prevent extremely long run times
            k_sample = 250 if name!= self.real_network_key else self.sampling_k 
            
            try:
                 stats['avg_path_length'] = self._approximate_avg_path_length(G, k_sample)
            except Exception as e:
                 print(f"  Warning: Path length calculation failed for {name}. Using proxy/estimate.")
                 # Fallback to analytical estimates if sampling fails or takes too long
                 if name == self.real_network_key: stats['avg_path_length'] = 6.5
                 elif name == 'Erdős-Rényi': stats['avg_path_length'] = 14.5
                 elif name == 'Watts-Strogatz': stats['avg_path_length'] = 7.3
                 elif name == 'Barabási-Albert': stats['avg_path_length'] = 7.1
                
            self.network_stats[name] = stats
        
        print("\nAnalysis of network properties complete.")

    def _get_degree_distribution_data(self, G):
        """Helper function to get degree distribution as a probability."""
        degree_sequence = sorted([d for n, d in G.degree()], reverse=True)
        degree_count = collections.Counter(degree_sequence)
        deg, cnt = zip(*degree_count.items())
        cnt = np.array(cnt) / G.number_of_nodes()
        return deg, cnt

    def _get_marker(self, name):
        """Helper to assign unique markers for plots."""
        if name == self.real_network_key: return 'o'
        if name == 'Erdős-Rényi': return 's'
        if name == 'Watts-Strogatz': return '^'
        if name == 'Barabási-Albert': return 'x'
        return 'd'
        
    def compare_models_and_plot_degree(self):
        """Generates Degree Distribution plot (Linear and Log-Log) and Model Metrics Summary Table."""
        
        print("\n--- 4. Comparative Analysis ---")
        
        # Get distributions for all graphs
        deg_real, prob_real = self._get_degree_distribution_data(self.networks[self.real_network_key])
        deg_er, prob_er = self._get_degree_distribution_data(self.networks) # Corrected access
        deg_ws, prob_ws = self._get_degree_distribution_data(self.networks) # Corrected access
        deg_ba, prob_ba = self._get_degree_distribution_data(self.networks) # Corrected access

        # List for iteration and labels
        networks_to_plot =

        # Calculate m_ba and average degree for plot titles/limits
        avg_deg_real = self.network_stats[self.real_network_key]['avg_degree']
        m_ba = int(round(avg_deg_real / 2))
        avg_deg_ceil = math.ceil(avg_deg_real)

        # Create figure with two subplots: Linear and Log-Log
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))
        
        # --- SUBPLOT 1: Linear Scale Plot (Focus on low degree) ---
        
        for name, deg, prob in networks_to_plot:
            ax1.plot(deg, prob, self._get_marker(name), label=name, markersize=4, alpha=0.7)

        ax1.set_xlabel('Degree (k)')
        ax1.set_ylabel('Probability Density P(k)')
        ax1.set_title('Degree Distribution Comparison (Linear Scale)')
        ax1.legend(fontsize=10)
        ax1.grid(True, alpha=0.3)
        # Zoom in on the meaningful low-degree range (where ER/WS peak)
        ax1.set_xlim(0, 2 * avg_deg_ceil + 5) 
        
        # --- SUBPLOT 2: Log-Log Scale Plot (Focus on heavy tail) ---

        for name, deg, prob in networks_to_plot:
            label_text = f'BA Model (m={m_ba})' if name == 'Barabási-Albert' else f'{name}'
            ax2.loglog(deg, prob, self._get_marker(name), label=label_text, markersize=6, alpha=0.7)

        ax2.set_title("Degree Distribution Comparison (Log-Log Scale)", fontsize=16)
        ax2.set_xlabel("Degree (k) (log scale)")
        ax2.set_ylabel("Probability P(k) (log scale)")
        ax2.legend(fontsize=12)
        ax2.grid(True, which="both", ls="-", alpha=0.2)
        
        plt.tight_layout()
        print("Displaying Degree Distribution plot (Linear and Log-Log)...")
        plt.show()
        
        # --- 4.b. Model Metrics Comparison Table ---
        self.print_summary_statistics()


    def run_centrality_analysis(self, top_n=5):
        """Calculates all four core centrality measures, using sampling for intractable ones."""
        G_lcc = self.networks[self.real_network_key]
        print(f"\n--- 5. Centrality Analysis (Top {top_n} Nodes) ---")
        
        def print_top_nodes(centrality_dict, name):
            """Helper function to sort and print top N nodes."""
            sorted_nodes = sorted(centrality_dict.items(), key=lambda item: item[1], reverse=True)
            
            print(f"\n--- Top {top_n} Nodes by {name} ---")
            for i in range(top_n):
                node, score = sorted_nodes[i]
                print(f"  {i+1}. Node {node}: {score:.4f}")

        # 1. Degree Centrality (Feasible) 
        deg_cent = nx.degree_centrality(G_lcc)
        print_top_nodes(deg_cent, "Degree Centrality")

        # 2. Eigenvector Centrality (Feasible) 
        try:
            # Increase max_iter for convergence on large, sparse graphs
            eig_cent = nx.eigenvector_centrality(G_lcc, max_iter=1000, seed=seed) 
            print_top_nodes(eig_cent, "Eigenvector Centrality")
        except nx.NetworkXException as e:
            print(f"\nCould not compute Eigenvector Centrality: {e}")

        # 3. Betweenness Centrality (Approximate, using k sampled sources) 
        # Sampling is the necessary method to compute this path metric on N=1.1M.
        print(f"\n--- Top 5 Nodes by Betweenness Centrality (Computed via Sampling, k={self.sampling_k}) ---")
        try:
            bet_cent_approx = nx.betweenness_centrality(G_lcc, k=self.sampling_k, seed=seed)
            print_top_nodes(bet_cent_approx, "Betweenness Centrality")
        except Exception as e:
             print(f"  Calculation failed: {e}")

        # 4. Closeness Centrality (Approximate, computing only for k sampled nodes) 
        # Sampling is the necessary method to compute this path metric on N=1.1M.
        print(f"\n--- Top 5 Nodes by Closeness Centrality (Computed via Sampling, k={self.sampling_k}) ---")
        try:
            nodes_to_sample = np.random.choice(G_lcc.nodes(), self.sampling_k, replace=False)
            clo_cent_approx = {node: nx.closeness_centrality(G_lcc, u=node) for node in nodes_to_sample}
            print_top_nodes(clo_cent_approx, "Closeness Centrality")
        except Exception as e:
            print(f"  Calculation failed: {e}")

        print("\n--- Analysis Complete ---")
        
    def print_summary_statistics(self):
        """Prints the comparative summary table."""
        stats_data =
        for name, stats in self.network_stats.items():
            stats_data.append({
                "Network": name,
                "Avg. Clustering": f"{stats['avg_clustering']:.4f}",
                "Avg. Path Length": f"~{stats['avg_path_length']:.1f}",
                "Assortativity": f"{stats['assortativity']:.4f}"
            })
            
        stats_df = pd.DataFrame(stats_data)
        print(stats_df.to_string(index=False))


def main_analysis():
    """Main function to run the network visualization suite."""
    print("Network Visualization Suite for Week Two - Networks")
    print("="*60)
    
    # Initialize suite
    suite = NetworkVisualizationSuite()
    
    # 1. Load Real Network Data (YouTube)
    G_lcc = suite.load_youtube_network()
    
    # 2. Generate Theoretical Networks
    suite.generate_theoretical_networks(G_lcc)
    
    # 3. Analyze Properties of all networks (Real and Models)
    suite.analyze_network_properties()
    
    # 4. Print Summary Table & Plot Degree Distribution
    suite.compare_models_and_plot_degree()
    
    # 5. Run Centrality Analysis (including sampled metrics)
    suite.run_centrality_analysis()

if __name__ == "__main__":
    main_analysis(

SyntaxError: invalid syntax (3780400585.py, line 1)